In [1]:
# Import Dependencies and modules
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from string import punctuation
from collections import Counter
from io import StringIO
from nltk.corpus import stopwords
import nltk
import glob
import errno
import os
import json

# Load Data

In [2]:
# Load each json file
with open('iphonex_digtrends.json') as f:
    iphonex_digtrends = json.load(f)

with open('iphonex_gizmodo.json') as f:
    iphonex_gizmodo = json.load(f)

with open('iphonex_techradar.json') as f:
    iphonex_techradar = json.load(f)

with open('S9_digtrends.json') as f:
    S9_digtrends = json.load(f)

with open('S9_gizmodo.json') as f:
    S9_gizmodo = json.load(f)

with open('S9_techradar.json') as f:
    S9_techradar = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'iphonex_digtrends.json'

# Clean Data

In [ ]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!

    text = str(text).replace("\n", "")
    text = str(text).replace("\t", "")
    text = str(text).replace("\\n", "")
    text = str(text).replace("\\t", "")
    text = str(text).replace("\\", "")
    text = str(text).replace("xa0", " ")
    text = str(text).replace("\'", "")
    text = re.sub("<p>", "", str(text))
    text = re.sub("</p>", "", str(text))
    text = re.sub("</a>", "", str(text))
    text = re.sub('<[^>]+>', "", str(text))
    text = str(text).replace("\\u2019", "")
    text = str(text).replace("\\u2013", "")
    text = str(text).replace("\\u2018", "")
    text = str(text).replace("\\u00a0", "")
    text = str(text).replace("\\u00a3", "")
    text = str(text).replace("\u2014", "")
    text = str(text).replace("\u201d", "")
    text = str(text).replace("\u201c", "")
    return text


In [ ]:
# Populate each JSON file into a data frame

iphonex_digtrends = pd.DataFrame.from_dict(iphonex_digtrends, orient='columns')
iphonex_gizmodo = pd.DataFrame.from_dict(iphonex_gizmodo, orient='columns')
iphonex_techradar = pd.DataFrame.from_dict(iphonex_techradar, orient='columns')
S9_digtrends = pd.DataFrame.from_dict(S9_digtrends, orient='columns')
S9_gizmodo = pd.DataFrame.from_dict(S9_gizmodo, orient='columns')
S9_techradar = pd.DataFrame.from_dict(S9_techradar, orient='columns')

In [ ]:
# Define function to clean text
def clean_text(df):
    # Convert lists to strings and remove brackets
    df['text'] = df['text'].astype(str)
    df['author'] = df['author'].astype(str)

    df['text'] = df['text'].map(lambda x: x.strip('[]'))
    df['author'] = df['author'].map(lambda x: x.strip('[]'))

    # Clean text
    df['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df['title'] = df['title'].apply(lambda x: text_cleaner(x))
    df['author'] = df['author'].apply(lambda x: text_cleaner(x))

    
# Put dataframes into a list to iterate through
dataframes = [iphonex_digtrends, iphonex_gizmodo, iphonex_techradar, S9_digtrends, S9_gizmodo, S9_techradar]

# Clean each Data Frame
for dataframe in dataframes:
    clean_text(dataframe)

In [ ]:
# Label all the rows in the dataframe for the phone that the article is talking about

iphones = [iphonex_digtrends, iphonex_gizmodo, iphonex_techradar]
s9s = [S9_digtrends, S9_gizmodo, S9_techradar]

for dataframe in iphones:
    dataframe['phone'] = 'IPhone X'
    
for dataframe in s9s:
    dataframe['phone'] = 'Samsung Galaxy S9'


In [ ]:
# Concat all the dataframes into one dataframe
all_frames = [iphonex_digtrends, iphonex_gizmodo, iphonex_techradar, S9_digtrends, S9_gizmodo, S9_techradar]
df = pd.concat(all_frames)


In [ ]:
# Visualize dataframe
df.head()

# Pre-Process Data for NLP

In [ ]:
# # Tokenize text
# df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
# df['title'] = df.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

# # Remove Stopwords, or keep it, might be important for aspect based semantics
# stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
# df['title'] = df['title'].apply(lambda x: [item for item in x if item not in stop])

# # Lowercase everything
# df['text'] = df['text'].astype(str)
# df['text'] = df['text'].apply(lambda x: x.lower())

# df['title'] = df['title'].astype(str)
# df['title'] = df['title'].apply(lambda x: x.lower())

# # remove all punctuations
# df['text'] = df['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
# df['title'] = df['title'].apply(lambda x: ''.join(c for c in x if c not in punctuation))

In [ ]:
from spacy.lang.en import English

spacy.load('en')
parser = English()

# Function to tokenize text
def tokenize(text):
    lda_tokens = []
    tokens  = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
# Download wordnet to find meaning of words, synonyms and antonyms
nltk.download('wordnet')

In [ ]:
from nltk.corpus import wordnet as wn

# Function to lemmatize and more words to their root
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
# Compile set of stopwords
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
text_data = []

# Prepare training set for LDA
tokens = df['text'].apply(lambda x: prepare_text_for_lda(x))

# Prepare Dataframe for later
df['text'] = df['text'].apply(lambda x: prepare_text_for_lda(x))

# Append tokenized text to list of tokenized data
null = tokens.apply(lambda x: text_data.append(x))

In [ ]:
### Clean Title Column

# Tokenize text
df['title'] = df.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

# Remove Stopwords, or keep it, might be important for aspect based semantics
stop = stopwords.words('english')
df['title'] = df['title'].apply(lambda x: [item for item in x if item not in stop])

# Lowercase everything
df['title'] = df['title'].astype(str)
df['title'] = df['title'].apply(lambda x: x.lower())

In [ ]:
# Visualize dataframe
df.head()

In [ ]:
from gensim import corpora

# Assemble tokenized text data into a dictionary
dictionary = corpora.Dictionary(text_data)

In [ ]:
# Create Bag of Words corpus from text data
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
from gensim import models

# Create TF-IDF vectors from our bag of words
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
import pickle

pickle.dump(corpus_tfidf, open('corpus_tfidf.pkl', 'wb'))
dictionary.save('dictionary.gensim')

# 3 Topics

In [ ]:
import gensim

NUM_TOPICS = 3

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model3.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

# 4 Topics

In [ ]:
import gensim

NUM_TOPICS = 4

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model4.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

# 5 Topics

In [ ]:
import gensim

NUM_TOPICS = 5

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model5.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

# 7 Topics

In [ ]:
import gensim

NUM_TOPICS = 7

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model7.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

# 10 Topics

In [ ]:
import gensim

NUM_TOPICS = 10

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model10.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

# pyLDAvis

In [ ]:
# Create Visualizations of topic clusters using pyLDAvis

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/corpus_tfidf.pkl', 'rb'))


In [ ]:
import pyLDAvis.gensim

# Display 3 topics
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)